In [2]:
import json
import requests
import websockets

# Apply public connect token

In [3]:
url = 'https://api.kucoin.com/api/v1/bullet-public'

In [4]:
x = requests.post(url)

In [5]:
cjson = json.loads(x.text)

In [6]:
endpoint = cjson['data']['instanceServers'][0]['endpoint']
token = cjson['data']['token']

# Subscription

In [7]:
sub = {
    "id": 1545910660739,
    "type": "subscribe",
    "topic": "/market/ticker:BTC-USDT",
    "response": True
}

# Single Websocket Connection

In [7]:
async with websockets.connect(endpoint+"?token="+token) as conn: # creates ws connection
    await conn.send(json.dumps(sub)) # sends ws message
    res = await conn.recv() # recieves ws message
    
    while True:
        res = await conn.recv()
        print(res)
    await conn.close()

{"id":"1545910660739","type":"ack"}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22467.8","bestAskSize":"0.04714862","bestBid":"22467.7","bestBidSize":"0.0006","price":"22468.8","sequence":"1631213019370","size":"0.000867","time":1655230739869}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22463.3","bestAskSize":"0.02213078","bestBid":"22463.2","bestBidSize":"0.50044","price":"22466","sequence":"1631213019815","size":"0.00234606","time":1655230739970}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22463.1","bestAskSize":"0.02357431","bestBid":"22463","bestBidSize":"0.0032424","price":"22466","sequence":"1631213020036","size":"0.00234606","time":1655230740070}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22463.1","bestAskSize":"0.4757051","bestBid":"22463","bestBidSize":"0.0032424","pri

{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22456.9","bestAskSize":"0.06671113","bestBid":"22456.8","bestBidSize":"0.62321641","price":"22456.9","sequence":"1631213022973","size":"0.01150821","time":1655230742970}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22456.9","bestAskSize":"0.06671113","bestBid":"22456.8","bestBidSize":"0.62321641","price":"22456.9","sequence":"1631213023021","size":"0.01150821","time":1655230743069}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22456.9","bestAskSize":"0.06671113","bestBid":"22456.8","bestBidSize":"0.62321641","price":"22456.9","sequence":"1631213023095","size":"0.01150821","time":1655230743168}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22456.9","bestAskSize":"0.07812945","bestBid":"22456.8","bestBidSize":"0.41088965","price":"22456.9","s

{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22456.9","bestAskSize":"0.00004","bestBid":"22456.8","bestBidSize":"0.04033687","price":"22454.9","sequence":"1631213026828","size":"0.01141832","time":1655230746169}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22456.9","bestAskSize":"0.00004","bestBid":"22456.8","bestBidSize":"0.28837183","price":"22454.9","sequence":"1631213026999","size":"0.01141832","time":1655230746270}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22456.9","bestAskSize":"0.00004","bestBid":"22456.8","bestBidSize":"1.00118495","price":"22454.9","sequence":"1631213027101","size":"0.01141832","time":1655230746369}}
{"type":"message","topic":"/market/ticker:BTC-USDT","subject":"trade.ticker","data":{"bestAsk":"22457.7","bestAskSize":"0.00035","bestBid":"22457.6","bestBidSize":"0.05699964","price":"22457.1","sequence":"16

CancelledError: 

# Multiplex Websocket Connection SPOT-FUTURE

In [8]:
multi = {"id": "1Jpg30DEdU", "type": "openTunnel", "newTunnelId": "bt1", "response": False}
# command used to open tunnel

In [9]:
multi_spotEth = {"id": "1JpoPamgFM",
             "type": "subscribe",
             "topic": "/market/ticker:ETH-USDT",
             "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

In [10]:
multi_futureEth = {"id": 1545910660740,                          
                "type": "subscribe",
                "topic": "/contractMarket/tickerV2:ETHUSDTM",
                "tunnelId": "bt1", "response": False}
# subscriptions to websocket with tunnelID attached

<h3>Multiplex Websocket Connection</h3>

In [ ]:
async with websockets.connect(endpoint+"?token="+token) as conn:
    await conn.send(json.dumps(multi))  # initial open tunel command
    res = await conn.recv()
    
    await conn.send(json.dumps(multi_spotEth)) # send tunnel subscription
    res = await conn.recv()
    await conn.send(json.dumps(multi_futureEth))  # send tunnel subscription
    res = await conn.recv()
    
    counter = 0
    spotPrice = 0
    futurePrice = 0
    while True:
        res = await conn.recv()
        vals = json.loads(res)

        if (vals['topic']) == '/contractMarket/tickerV2:ETHUSDTM':
#             print(f"Future Price is: {vals['data']['bestAskPrice']}")
            futurePrice = float(vals['data']['bestAskPrice'])

        elif (vals['topic']) == '/market/ticker:ETH-USDT':
#             print(f"Spot Price is: {vals['data']['bestAsk']}")
            spotPrice = float(vals['data']['bestAsk'])
    
#         print(spotPrice - futurePrice)
        
        if spotPrice - futurePrice > 0.8 and futurePrice != 0 and spotPrice != 0:
            print('Long Future and Short Spot')
            
        elif futurePrice - spotPrice > 0.8 and futurePrice != 0 and spotPrice != 0:
            print('Long Spot and Short Future')
            
        counter += 1
    eth_tunnel = {"id": "1JpoPamgFM", "type": "closeTunnel", "tunnelId": "bt1", "response": False}            
    btc_tunnel = {"id": "1JpoPamgFN", "type": "closeTunnel", "tunnelId": "bt1", "response": False}
    await conn.send(json.dumps(eth_tunnel))
    await conn.send(json.dumps(btc_tunnel))
    await conn.close()

# Triangle Arbitrage

In [13]:
async with websockets.connect(endpoint+"?token="+token) as conn:
    await conn.send(json.dumps(multi))  # initial open tunel command
    res = await conn.recv()
    
    await conn.send(json.dumps(multi_spotEth)) # send tunnel subscription
    res = await conn.recv()
#     await conn.send(json.dumps(multi_futureEth))  # send tunnel subscription
#     res = await conn.recv()
    
    counter = 0
    spotPrice = 0
    futurePrice = 0
    while True and counter < 1:
        res = await conn.recv()
        vals = json.loads(res)

#         if (vals['topic']) == '/contractMarket/tickerV2:ETHUSDTM':
#             print(f"Future Price is: {vals['data']['bestAskPrice']}")
#             futurePrice = float(vals['data']['bestAskPrice'])

        if (vals['topic']) == '/market/ticker:ETH-USDT':
            print(vals)
            
            
        counter += 1
    eth_tunnel = {"id": "1JpoPamgFM", "type": "closeTunnel", "tunnelId": "bt1", "response": False}            
    btc_tunnel = {"id": "1JpoPamgFN", "type": "closeTunnel", "tunnelId": "bt1", "response": False}
    await conn.send(json.dumps(eth_tunnel))
    await conn.send(json.dumps(btc_tunnel))
    await conn.close()

{'type': 'message', 'tunnelId': 'bt1', 'topic': '/market/ticker:ETH-USDT', 'subject': 'trade.ticker', 'data': {'bestAsk': '1227.09', 'bestAskSize': '6.2468595', 'bestBid': '1227.08', 'bestBidSize': '0.1248', 'price': '1227.09', 'sequence': '1629525353061', 'size': '0.007169', 'time': 1655230779442}}


# Trying to Long the underpriced and short the overpriced